In [8]:
import os
import shutil
from sklearn.model_selection import train_test_split

base = "dataset/dogs-vs-cats"
cats = os.listdir(f"{base}/train cats")
dogs = os.listdir(f"{base}/train dogs")

# Crear carpetas nuevas
for part in ["train", "val"]:
    os.makedirs(f"{base}/{part}/cats", exist_ok=True)
    os.makedirs(f"{base}/{part}/dogs", exist_ok=True)

# Split
cats_train, cats_val = train_test_split(cats, test_size=0.2, random_state=42)
dogs_train, dogs_val = train_test_split(dogs, test_size=0.2, random_state=42)

# Copiar gatos
for f in cats_train:
    shutil.move(f"{base}/train cats/{f}", f"{base}/train/cats/{f}")

for f in cats_val:
    shutil.move(f"{base}/train cats/{f}", f"{base}/val/cats/{f}")

# Copiar perros
for f in dogs_train:
    shutil.move(f"{base}/train dogs/{f}", f"{base}/train/dogs/{f}")

for f in dogs_val:
    shutil.move(f"{base}/train dogs/{f}", f"{base}/val/dogs/{f}")

print("✔ División correcta en TRAIN y VALIDATION.")


✔ División correcta en TRAIN y VALIDATION.


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (150,150)
batch = 32

train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "dataset/dogs-vs-cats/train",
    target_size=image_size,
    batch_size=batch,
    class_mode="binary"
)

val_data = val_gen.flow_from_directory(
    "dataset/dogs-vs-cats/val",
    target_size=image_size,
    batch_size=batch,
    class_mode="binary"
)


Found 8604 images belonging to 2 classes.
Found 2152 images belonging to 2 classes.


In [10]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Cargar MobileNetV2 con pesos ImageNet
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(150,150,3))
base_model.trainable = False  # congelar

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 75, 75, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 75, 75, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)             

In [11]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


Epoch 1/5
269/269 [==============================] - 52s 187ms/step - loss: 0.1239 - accuracy: 0.9510 - val_loss: 0.1357 - val_accuracy: 0.9475
Epoch 2/5
269/269 [==============================] - 51s 191ms/step - loss: 0.0860 - accuracy: 0.9670 - val_loss: 0.1442 - val_accuracy: 0.9452
Epoch 3/5
269/269 [==============================] - 51s 191ms/step - loss: 0.0618 - accuracy: 0.9761 - val_loss: 0.1075 - val_accuracy: 0.9526
Epoch 4/5
269/269 [==============================] - 53s 196ms/step - loss: 0.0483 - accuracy: 0.9827 - val_loss: 0.1274 - val_accuracy: 0.9549
Epoch 5/5
269/269 [==============================] - 52s 193ms/step - loss: 0.0336 - accuracy: 0.9890 - val_loss: 0.1403 - val_accuracy: 0.9503


In [12]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

image_size = (150,150,3)

# 1. Cargar base pre-entrenada
base = MobileNetV2(
    input_shape=image_size,
    include_top=False,
    weights="imagenet"
)
base.trainable = False   # congelamos capas

# 2. Añadir cabeza custom
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base.input, outputs=output)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 75, 75, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 75, 75, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 75, 75, 32)   0           ['bn_Conv1[0][0]']         

In [13]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


Epoch 1/5
269/269 [==============================] - 53s 193ms/step - loss: 0.1436 - accuracy: 0.9444 - val_loss: 0.1225 - val_accuracy: 0.9484
Epoch 2/5
269/269 [==============================] - 51s 191ms/step - loss: 0.0993 - accuracy: 0.9619 - val_loss: 0.1179 - val_accuracy: 0.9512
Epoch 3/5
269/269 [==============================] - 52s 193ms/step - loss: 0.0862 - accuracy: 0.9669 - val_loss: 0.1093 - val_accuracy: 0.9545
Epoch 4/5
269/269 [==============================] - 58s 216ms/step - loss: 0.0757 - accuracy: 0.9689 - val_loss: 0.1130 - val_accuracy: 0.9549
Epoch 5/5
269/269 [==============================] - 56s 209ms/step - loss: 0.0691 - accuracy: 0.9740 - val_loss: 0.1065 - val_accuracy: 0.9559


In [14]:
model.save("cats_vs_dogs_model.h5")


In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150,150))
    x = image.img_to_array(img) / 255.0
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)[0][0]
    return "DOG" if pred > 0.5 else "CAT"
